In [5]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd
"""
Parameter Correction Module
----------------------------
Performs minimal weighted corrections on a given parameter vector
to satisfy linear constraints using constrained optimization.
Designed for scientific or engineering use, but written to be clear
and understandable for all users.
"""

def correct_parameters(
    original_values: np.ndarray,
    constraint_matrix: np.ndarray,
    target_values: np.ndarray,
    weights: np.ndarray | None = None,
    max_correction: float = 25.0,
    constraint_tolerance: float = 9.0,
) -> tuple[pd.DataFrame, pd.DataFrame, bool]:
    """
    Apply minimal weighted corrections Δx to original_values so that:
        A·(x + Δx) ≈ b
    holds within ±constraint_tolerance, with each Δx bounded by ±max_correction.

    Returns two DataFrames (results and residuals) and a success flag.
    """
    n = original_values.size
    m = constraint_matrix.shape[0]

    # Default equal weighting if user doesn't provide specific importance weights
    weights = np.ones(n) if weights is None else weights

    # Compute how far current values are from target constraints
    rhs_residual = target_values - constraint_matrix @ original_values

    # Objective: minimize total weighted correction magnitude
    def objective(delta):
        return np.sum(weights * delta**2)

    # Each variable correction is bounded for realism
    bounds = [(-max_correction, max_correction)] * n

    # Build two inequality constraints per row in A to enforce tolerance limits
    constraints = []
    for i in range(m):
        Ai = constraint_matrix[i, :]
        ri = rhs_residual[i]
        constraints += [
            # Enforce: A·Δx - r ≤ tolerance
            {'type': 'ineq', 'fun': lambda delta, Ai=Ai, ri=ri: constraint_tolerance - (Ai @ delta - ri)},
            # Enforce: -(A·Δx - r) ≤ tolerance
            {'type': 'ineq', 'fun': lambda delta, Ai=Ai, ri=ri: constraint_tolerance + (Ai @ delta - ri)},
        ]

    # Solve quadratic optimization problem using Sequential Least Squares Programming
    result = minimize(
        objective,
        x0=np.zeros(n),
        bounds=bounds,
        constraints=constraints,
        method="SLSQP"
    )

    # Apply optimal corrections
    delta = result.x
    corrected_values = original_values + delta

    # Measure how well final solution satisfies constraints
    final_residuals = constraint_matrix @ corrected_values - target_values
    residual_norm = np.linalg.norm(final_residuals, ord=np.inf)  # largest absolute error

    # Summarize correction per variable
    df_results = pd.DataFrame({
        "Variable": [f"X{i+1}" for i in range(n)], #X = H or S
        "Original": original_values,
        "Correction": delta,
        "Corrected": corrected_values
    })

    # Summarize constraint satisfaction
    df_residuals = pd.DataFrame({
        "Constraint": [f"C{i+1}" for i in range(m)],
        "Residual": final_residuals
    })

    # Quick console feedback for the user
    print("\nResiduals:")
    print(df_residuals)
    print(f"Max residual (∞-norm): {residual_norm:.4f}")
    print("✅ Optimization successful." if result.success else "❌ Optimization failed. Try relaxing parameters.")

    return df_results, df_residuals, result.success


def run_enthalpic_corrections():
    """Demonstrates correction for reaction enthalpies."""
    # Initial individual reaction enthalpies (17 reaction enthalpies in total), obtained from Table S6
    H_orig = np.array([
        -1.93, 5.79, 43.42, -13.51, -13.51, 74.30, 43.42,
        -123.51, -23.16, 68.51, -2.89, -1.93, 92.63, -32.81,
        101.03, -114.82, 23.78
    ])

    # 4 constraints × 17 Hs (17 elementary reactions)
    A = np.zeros((4, 17))
    b = np.array([260.44, 260.44, 35.05, 35.05]) # Overall gas phase rxn enthalpies for D1, D2, W1 and W2

    # Custom constraint definitions - overall rxn enthalpy definitions for D1, D2 and W1, W2 pathways
    A[0, [0, 1, 2, 3, 4, 5, 8, 9, 10, 11, 12]] = 1 #D1
    A[0, 5] += 1 #D2
    A[1, [0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 12]] = 1 #D2
    A[1, 5] += 1 #D2
    A[2, [11, 12, 15, 16, 5]] = [1, 1, 1, 1, -1] #W1
    A[3, [12, 13, 14, 15, 16, 5, 10]] = [1, 1, 1, 1, 1, -1, -1] #W2

    # Higher weights = more resistant to change
    weights = np.array([10, 1, 0.1, 0.1, 0.1, 1, 0.01, 0.01, 1, 0.1,
                        0.1, 10, 1, 1, 0.1, 0.1, 0.1]) #weights for every elementary reaction, also available in Table S8

    df, df_residuals, success = correct_parameters(H_orig, A, b, weights, max_correction=25)
    print(df) #this data frame is used to create Table S9
    print(f"Success: {success}")
    return df


def run_entropic_corrections():
    """Demonstrates correction for reaction entropies"""
    #S original - Set of initial individual reaction entropies (17 rxn entropies in total), obtained from Table S6
    S_orig = np.array([
        -112.45, -37.41, -1.77, -10.32, 0.79, 102.00, 6.78, 3.75,
        18.26, 135.76, -101.25, -112.45, 2.31, -30.51, 57.41,
        -5.34, 18.32
    ])

    A = np.zeros((4, 17))
    b = np.array([284.60, 284.60, 32.03, 32.03])  #Overall gas phase rxn entropies for D1, D2, W1 and W2
    # Custom constraint definitions - overall rxn entropy definitions for D1, D2 and W1, W2 pathways
    A[0, [0, 1, 2, 3, 4, 5, 8, 9, 10, 11, 12]] = 1 #D1
    A[0, 5] += 1 #D1
    A[1, [0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 12]] = 1 #D2
    A[1, 5] += 1 #D2
    A[2, [11, 12, 15, 16, 5]] = [1, 1, 1, 1, -1] #W1
    A[3, [12, 13, 14, 15, 16, 5, 10]] = [1, 1, 1, 1, 1, -1, -1] #W2

    weights = np.array([0.1, 1, 0.1, 0.1, 0.1, 0.1, 0.01, 0.01, 1, 0.1,
                        0.1, 0.1, 1, 1, 0.1, 0.1, 0.1]) #weights for every elementary reaction, also available in Table S8

    df, df_residuals, success = correct_parameters(S_orig, A, b, weights, max_correction=100)
    print(df) #this data frame is used to create Table S10
    print(f"Success: {success}")
    return df


if __name__ == "__main__":
    # Run both corrections when executed directly
    print("Running enthalpy corrections...")
    run_enthalpic_corrections()
    print("\nRunning entropic corrections...")
    run_entropic_corrections()


Running enthalpy corrections...

Residuals:
  Constraint  Residual
0         C1       9.0
1         C2      -9.0
2         C3      -9.0
3         C4       9.0
Max residual (∞-norm): 9.0000
✅ Optimization successful.
   Variable  Original  Correction   Corrected
0        X1     -1.93   -0.075107   -2.005107
1        X2      5.79   -0.750981    5.039019
2        X3     43.42   -7.509752   35.910248
3        X4    -13.51   -7.509752  -21.019752
4        X5    -13.51   -8.323104  -21.833104
5        X6     74.30  -25.000000   49.300000
6        X7     43.42    8.132332   51.552332
7        X8   -123.51    8.132332 -115.377668
8        X9    -23.16   -0.832233  -23.992233
9       X10     68.51   -7.509766   61.000234
10      X11     -2.89   24.240696   21.350696
11      X12     -1.93    2.624133    0.694133
12      X13     92.63   23.065867  115.695867
13      X14    -32.81   -3.175171  -35.985171
14      X15    101.03  -25.000000   76.030000
15      X16   -114.82   25.000000  -89.820000
16